In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import load, Loader
from json import dump
import subprocess

## User-Defined Parameters

These parameters should be set by the user. `MFLOPS` is determined by the Linpack Benchmark test as stated in the README, and `output_path` should point to the directory containing data generated by the experiment. If you followed along with the README, no change to `output_path` should be necessary.

In [6]:
MFLOPS = 1250
output_path = Path() / "complexity_results"

## Pre-Defined Parameters

It is not necessary for you to modify these parameters, and the program may not work if you do change them.

In [7]:
FLOPS = MFLOPS * 10E6
raw_path = output_path / "raw_data"
actual_path = raw_path / "actual_time.yaml"
theoretical_path = raw_path / "theoretical_time.yaml"
metadata_path = output_path / "metadata.yaml"
mem_folder_path = raw_path / "memory_output"
data_dump_path = output_path / "processed_output"
output_fnames = ("actual_runtime.csv", "theoretical_runtime.csv", "bytes.csv", "all_fields.json")
MODELS = ["tf-necd", "tf-cod", "pytorch-qrcp", "pytorch-qr", "pytorch-svd", "pytorch-svddc", "sklearn-svddc"]

## Loading Metadata

In [8]:
with open(metadata_path, "r") as f:
    metadata = load(f, Loader=Loader)
    
metadata

{'dataset_shape': '10000 x 10',
 'failed_regs': [],
 'failed_regs_exceptions': [],
 'reg_names': ['tf-necd',
  'tf-cod',
  'pytorch-qrcp',
  'pytorch-qr',
  'pytorch-svd',
  'pytorch-svddc',
  'sklearn-svddc'],
 'repeat': 10,
 'rows_in_experiment': [10, 31, 100, 316, 1000, 3162, 10000],
 'timer_method': 'process in nanoseconds'}

## Loading Actual Times

In [9]:
with open(actual_path, "r") as f:
    actual_times = load(f, Loader=Loader)
    
actual_times_dict = {reg_name: {row_count: np.mean([nanoseconds for rc, nanoseconds in actual_times if row_count == rc]) for row_count, _ in actual_times} for reg_name, actual_times in actual_times.items()}

## Loading Theoretical Flop Counts

In [10]:
with open(theoretical_path, "r") as f:
    theoretical_flops = load(f, Loader=Loader)
    
theoretical_times_dict = {reg_name: {row_count: (flops / FLOPS) * 10E9 for row_count, flops in theoretical_flops} for reg_name, theoretical_flops in theoretical_flops.items()}

## Loading / Processing Binary Memory Files

In [11]:
make_mempath = lambda reg_name, row_count, iter, ext: mem_folder_path / f"{'mem' if ext == 'bin' else 'memray-csv-mem'}_{reg_name}_{row_count}_{iter}.{ext}"

mem_dict = {}
for reg_name in MODELS:
    mem_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        runs = []
        for iter in range(metadata["repeat"]):
            # use memray transform to actually create csv
            input_path = make_mempath(reg_name, row_count, iter, "bin")
            print(f"{input_path =}")
            subprocess.run(["memray", "transform", "csv", input_path, "-f"])
            
            # load csv normally, sum all mem usage
            output_path = make_mempath(reg_name, row_count, iter, "csv")
            single_mem_df = pd.read_csv(output_path, header=0, index_col=None)
            bytes_used = single_mem_df["size"].sum()
            runs.append(bytes_used)
            
        mem_dict[reg_name][row_count] = np.mean(runs)

input_path =PosixPath('complexity_results/raw_data/memory_output/mem_tf-necd_10_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote complexity_results/raw_data/memory_output/memray-csv-mem_tf-necd_10_0.csv
input_path =PosixPath('complexity_results/raw_data/memory_output/mem_tf-necd_10_1.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote complexity_results/raw

## Aggregate and Output Data

In [12]:
json_prep_dict = {}
for reg_name in MODELS:
    json_prep_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        json_prep_dict[reg_name][row_count] = {
            "real_runtime": actual_times_dict[reg_name][row_count],
            "theoretical_runtime": theoretical_times_dict[reg_name][row_count],
            "bytes": mem_dict[reg_name][row_count]
        }

df_real_runtime = pd.DataFrame(actual_times_dict)
df_theoretical_runtime = pd.DataFrame(theoretical_times_dict)
df_memory = pd.DataFrame(mem_dict)

data_dump_path.mkdir(exist_ok=True)

df_real_runtime.to_csv(data_dump_path / output_fnames[0], header=True, index=True)
df_theoretical_runtime.to_csv(data_dump_path / output_fnames[1], header=True, index=True)
df_memory.to_csv(data_dump_path / output_fnames[2], header=True, index=True)

with open(data_dump_path / output_fnames[3], "w") as f:
    dump(json_prep_dict, f)
    
print(f"All data has been placed at: {data_dump_path.resolve()}")

All data has been placed at: /Users/joshuaelms/Desktop/github_repos/are-they-what-they-claim-supplementary-materials/complexity_exper/analysis/complexity_results/processed_output
